In [1]:
import requests
import re
from datetime import datetime

In [2]:
cik_num = '789019' # currently AAPL (Apple Inc)

In [3]:
base_url = r"https://www.sec.gov/Archives/edgar/data"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'
}

In [4]:
# Get the filings in JSON format
filings_url = f"{base_url}/{cik_num}/index.json"
content = requests.get(filings_url, headers=HEADERS)
decoded_content = content.json()

In [5]:
# Define the desired form types
desired_form_types = ['10-K', '10-Q']

# Get the current year
current_year = datetime.now().year

In [11]:
# Initialize a list to store the relevant filings
relevant_filings = []

# Iterate through the filings
for filing_number in decoded_content['directory']['item'][:1]:
    filing_num = filing_number['name']
    
    # Define the filing URL in JSON format
    filing_url = f"{base_url}/{cik_num}/{filing_num}/index.json"

    # Get the documents submitted for that filing
    content = requests.get(filing_url, headers=HEADERS)
    document_content = content.json()

    # Iterate through the documents in the filing
    for document in document_content['directory']['item']:
        document_name = document['name']

        # Check if the document is a ".txt" file
        if document_name.endswith('.txt'):
            # Define the URL of the ".txt" file
            txt_file_url = f"{base_url}/{cik_num}/{filing_num}/{document_name}"

            # Get the contents of the ".txt" file
            txt_file_content = requests.get(txt_file_url, headers=HEADERS).text

            # Extract the filing date using a regular expression
            match_date = re.search(r'(\d{4})(\d{2})(\d{2})', document_name)
            print(document_content)
            if match_date:
                year, month, day = match_date.groups()
                filing_date = f"{year}-{month}-{day}"
                filing_year = int(year)

                # Check if the filing is a 10-K or 10-Q for the current year
                if (
                    filing_year == current_year
                    and any(form_type in txt_file_content for form_type in desired_form_types)
                ):
                    relevant_filings.append({
                        "date": filing_date,
                        "form_type": re.search(r'FORM TYPE:[^\n]*\n([^\n]*)', txt_file_content).group(1).strip(),
                        "file_url": txt_file_url,
                    })

{'directory': {'item': [{'last-modified': '2023-10-26 12:52:13', 'name': '0001214659-23-013902-index-headers.html', 'type': 'text.gif', 'size': ''}, {'last-modified': '2023-10-26 12:52:13', 'name': '0001214659-23-013902-index.html', 'type': 'text.gif', 'size': ''}, {'last-modified': '2023-10-26 12:52:13', 'name': '0001214659-23-013902.txt', 'type': 'text.gif', 'size': ''}, {'last-modified': '2023-10-26 12:52:13', 'name': 'asyousowlogo.jpg', 'type': 'image2.gif', 'size': '8466'}, {'last-modified': '2023-10-26 12:52:13', 'name': 'p1026230px14a6g.htm', 'type': 'text.gif', 'size': '62058'}], 'name': '/Archives/edgar/data/789019/000121465923013902', 'parent-dir': '/Archives/edgar/data/789019'}}


In [16]:
print(relevant_filings)

[]
